In [7]:
import Random
using Distributions
using Statistics
using DelimitedFiles
using Dates

In [8]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_mRNA_fb_rates! (generic function with 1 method)

In [9]:
function data_collection(initialstate::Vector{Int64},params::Vector{Float64},outcomes,storage,numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG,ntherm)
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,storage,updatestorage!,rng,numsteps,maxsteps,ntherm)
    return storage,endstate,steps,weightsum
end

function reach_balance_test(initialstate,params,outcomes,numsteps,maxsteps,rng,flux_thres,cov_thres,tries,ntherm)
    statevector = initialstate
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    calcstor = BalanceEqData_Fy()
    fluxcheck = [0,0]
    cbvals = [0,0]
    
    while (trynum < tries) && (!thresholdmet)
        calcstor,statevector,steps,lastsimtime = data_collection(statevector,params,outcomes,calcstor,numsteps,maxsteps,rng,ntherm)
        weightsum = calcstor.weightsum
        fluxcheck = [relative_error(calcstor.meanRpX,calcstor.meanRmX),relative_error(calcstor.meanRpY,calcstor.meanRmY)]
        if (fluxcheck[1] < flux_thres) &&  (fluxcheck[2] < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/weightsum,calcstor.CXRmX/weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/weightsum,calcstor.CXRpX/weightsum) 
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/weightsum,calcstor.CYRmY/weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/weightsum,calcstor.CYRpY/weightsum) 
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/weightsum,calcstor.CXRmY/weightsum,
                            0,calcstor.CYRpX/weightsum,calcstor.CXRpY/weightsum) 
            cbvals = [cbxx_comp,cbyy_comp,cbyx_comp]
            thresholdmet = all(cbvals .< cov_thres)               
        end
        trynum += 1
        totalsteps += steps
        if maxsteps>100000
            maxsteps = maxsteps ÷ 2
        end
    end
return calcstor,totalsteps,trynum,fluxcheck,cbvals
end

reach_balance_test (generic function with 1 method)

Run some number of simulations, storing the used parameters+simulation constraints. Save each simulation's output into a big CSV. Finally, go through the results and calculate the perturbation data

Generate truly independent perts: cartesian product of all the values you want to multiply

In [48]:
ds = LinRange(0,0.6,10)

ipset = []
for d in ds
    lambdas = [1]
    betaxs = [1,1.2,0.8]
    gammas = [1,1+d,1-d]
    betays = [1]
    ns=[1]
    ks=[1]
    independentperts = [[d_l,d_bx,d_g,d_by,d_n,d_k] for d_l in lambdas for d_bx in betaxs for d_g in gammas for d_by in betays for d_n in ns for d_k in ks]
    append!(ipset,[repeat(independentperts,9)])
end

ns = [-1,-0.5,0.1,0.5,1.]
paramsets = [[200.,5.,200.,1.,n,100.] for n in ns]

10-element Vector{Any}:
 [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]  …  [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0]]
 [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0666666666666667, 1.0, 1.0, 1.0], [1.0, 1.0, 0.9333333333333333, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0, 1.0, 1.0, 1.0], [1.0, 1.2, 1.0666666666666667, 1.0, 1.0, 1.0], [1.0, 1.2, 0.9333333333333333, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0, 1.0, 1.0, 1.0], [1.0, 0.8, 1.0666666666666667, 1.0, 1.0, 1.0]

In [49]:
folder = "yfb_sims_lownoise_"*Dates.format(now(),"yyyy-udd_HHMM")*"/"
mkdir(folder)
is = [10,10]
statevector = copy(is)
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(1e5)
maxsteps = Int(1e8)
fluxt = 0.02
covt = 0.05
tries = 150
ntherm = Int(10e5)
adder = 0
for (a,ips) in enumerate(ipset)
    pertsetcov = cov(ips)
    open(folder*"perturbationset_$(a)_covs.tsv","w") do io
        writedlm(io,pertsetcov)
    end
    for (i,params) in enumerate(paramsets)
        pertparams = copy(params)
        open(folder*"pertset_$(a)_paramset_$(i)_settings.tsv", "w") do io
            titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
            data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
            writedlm(io,reshape(titles,1,length(titles)))
            writedlm(io,reshape(data,1,length(data)))
        end;
        open(folder*"pertset_$(a)_paramset_$(i)_simresults.tsv", "w") do io
            titles = ["sim #","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams","Fy"]
            writedlm(io, reshape(titles,1,length(titles)))
        end;
        for (j,perturbation) in enumerate(ips)
            statevector .= is
            pertparams .*= perturbation
            rng = Random.Xoshiro()
            calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
            open(folder*"pertset_$(a)_paramset_$(i)_simresults.tsv", "a+") do io
                data = [j,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams,calcstor.meanF_y]
                writedlm(io, reshape(data,1,length(data)))
            end;
            pertparams ./= perturbation
        end
    end
end

4-element Vector{Float64}:
 -2.0
 -1.366666666666667
 -0.7333333333333334
 -0.1